In [1]:
from naat_core.data_files import GLOBAL_LITIGATION_CASES_PATH
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv(GLOBAL_LITIGATION_CASES_PATH)
df.describe()

,ID,Filing Year
count,790.000000,636.000000
mean,11316.805063,2017.520440
std,6287.496942,4.981341
min,2204.000000,1994.000000
25%,2853.250000,2015.000000
50%,12448.500000,2019.000000
75%,17021.750000,2021.000000
max,19893.000000,2023.000000


In [7]:
df.columns

Index(['Title', 'ID', 'Case Permalink', 'Case Categories', 'Jurisdictions',
       'Principal Laws', 'Summary', 'Reporter Info or Case Number',
       'Filing Year', 'Status', 'Core Object', 'columns_concatenations'],
      dtype='object')

In [11]:
df.dtypes

Title                                   object
ID                                       int64
Case Permalink                          object
Case Categories                         object
Jurisdictions                           object
Principal Laws                          object
Summary                                 object
Reporter Info or Case Number            object
Filing Year                     datetime64[ns]
Status                                category
Core Object                             object
columns_concatenations                  object
dtype: object

In [4]:
def preprocess_fields(value):
    if isinstance(value, float):
        return []
    elif value:
        value = value.replace("&amp;", "&")
        values = value.split("|")
        return values
    else:
        return []

In [45]:
a = [1,2,3,4,5]
b = [0]
list(set(a) & set(b))

[]

In [9]:

df["Jurisdictions"] = df["Jurisdictions"].apply(preprocess_fields)
df["Case Categories"] = df["Case Categories"].apply(preprocess_fields)
df["Principal Laws"] = df["Principal Laws"].apply(preprocess_fields)
df["Filing Year"] = pd.to_datetime(df["Filing Year"], format="%Y")

df["columns_concatenations"] = df["Title"] + "\n" + df["Summary"] + "\n" + df["Core Object"]

df["Status"] = df["Status"].astype("category")

In [68]:
df["ID"]

0       2204
1       2504
2       2505
3       2508
4       2510
       ...  
785    19878
786    19879
787    19880
788    19892
789    19893
Name: ID, Length: 790, dtype: int64

In [66]:
def is_column_list(serie):
    is_column_array = (
        serie.sample(100).apply(type).mode(0).astype(str) == "<class 'list'>"
    )
    return is_column_array.values[0]


def get_column_list_categories(serie: pd.Series):
    unique_categories = list(
        set(serie.fillna("NA").explode().astype(str).to_list())
    )
    return sorted(unique_categories)

def search_in_column_list(df, column, queries):
    mask = df[column].apply(lambda x: len(set(queries) & set(x)) > 0)
    return df[mask]


In [67]:
get_column_list_categories(df["Principal Laws"])

['172/1966)',
 '1858 Treaty of Limits (between Nicaragua and Costa Rica)',
 '187/2009)',
 '257/2016',
 '445/2005)',
 'Aarhus Convention on Access to Environmental Information',
 'Act Against Unfair Competition>Article 5a',
 'African Charter on Human and Peoples’ Rights',
 'Agreement on Subsidies and Countervailing Measures (SCM Agreement)',
 'Agreement on Trade-Related Investment Measures (TRIMs Agreement)',
 'American Convention on Human Rights',
 'American Convention on Human Rights>San Salvador Protocol',
 'American Declaration of the Rights and Duties of Man',
 'Argentina>Constitution',
 'Argentina>Environmental Law to Control Burning Activities No. 23.919.',
 'Argentina>General Environmental Law No. 25.675',
 'Argentina>Law No. 22351',
 'Argentina>Law No. 23094',
 'Argentina>Law No. 26.815',
 'Argentina>Law No. 26562',
 'Argentina>Law No. 27520/2019 (Minimum Budgets for Adaptation and Mitigation)',
 'Argentina>Law No. 27566',
 'Argentina>NDC',
 'Atlantic Forest Law',
 'Australia',

In [24]:
is_column_list(df["Case Categories"])

True

In [51]:
search_in_column_list(df,"Principal Laws", set("Australia"))

,Title,ID,Case Permalink,Case Categories,Jurisdictions,Principal Laws,Summary,Reporter Info or Case Number,Filing Year,Status,Core Object,columns_concatenations


{'172/1966)',
 '1858 Treaty of Limits (between Nicaragua and Costa Rica)',
 '187/2009)',
 '257/2016',
 '445/2005)',
 'Aarhus Convention on Access to Environmental Information',
 'Act Against Unfair Competition>Article 5a',
 'African Charter on Human and Peoples’ Rights',
 'Agreement on Subsidies and Countervailing Measures (SCM Agreement)',
 'Agreement on Trade-Related Investment Measures (TRIMs Agreement)',
 'American Convention on Human Rights',
 'American Convention on Human Rights>San Salvador Protocol',
 'American Declaration of the Rights and Duties of Man',
 'Argentina>Constitution',
 'Argentina>Environmental Law to Control Burning Activities No. 23.919.',
 'Argentina>General Environmental Law No. 25.675',
 'Argentina>Law No. 22351',
 'Argentina>Law No. 23094',
 'Argentina>Law No. 26.815',
 'Argentina>Law No. 26562',
 'Argentina>Law No. 27520/2019 (Minimum Budgets for Adaptation and Mitigation)',
 'Argentina>Law No. 27566',
 'Argentina>NDC',
 'Atlantic Forest Law',
 'Australia',

# pandas_types.is_array_like
# pandas_types.is_bool
# pandas_types.is_dict_like
# pandas_types.is_list_like
# pandas_types.is_named_tuple